# Classification - Women's E-Commerce Clothing Reviews

In [1]:
!apt-get update

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.4.0/spark-2.4.0-bin-hadoop2.7.tgz
!tar -xvf spark-2.4.0-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.0-bin-hadoop2.7"
import findspark
findspark.init()

spark-2.4.0-bin-hadoop2.7/
spark-2.4.0-bin-hadoop2.7/python/
spark-2.4.0-bin-hadoop2.7/python/setup.cfg
spark-2.4.0-bin-hadoop2.7/python/pyspark/
spark-2.4.0-bin-hadoop2.7/python/pyspark/resultiterable.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/python/
spark-2.4.0-bin-hadoop2.7/python/pyspark/python/pyspark/
spark-2.4.0-bin-hadoop2.7/python/pyspark/python/pyspark/shell.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/heapq3.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/join.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/version.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/rdd.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/java_gateway.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/find_spark_home.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/_globals.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/worker.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/accumulators.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/mllib/
spark-2.4.0-bin-hadoop2.7/python/pyspark/mllib/feature.py
spark-2.4.0-bin-hadoop2.7/python/pyspark

In [3]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [4]:
%cd '/content/gdrive/My Drive/Colab Notebooks/LDS9/thi/'

/content/gdrive/My Drive/Colab Notebooks/LDS9/thi


In [5]:
import pandas as pd

In [6]:
import pyspark

In [7]:
from pyspark import SparkContext
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [8]:
from pyspark.sql import SQLContext 

In [9]:
sc = SparkContext(master = "local", appName= "Ex1")

In [10]:
spark = SparkSession(sc)

# Read Data

In [11]:
pdf = pd.read_excel('womens-ecommerce-clothing-reviews/Womens_Clothing_E_Commerce_Reviews.xlsx', 
                               sheet_name='Reviews', index_col=0)

In [12]:
pdf.head()

,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


In [13]:
pdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23481 entries, 0 to 23480
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Clothing ID              23481 non-null  int64 
 1   Age                      23481 non-null  int64 
 2   Title                    19671 non-null  object
 3   Review Text              22636 non-null  object
 4   Rating                   23481 non-null  int64 
 5   Recommended IND          23481 non-null  int64 
 6   Positive Feedback Count  23481 non-null  int64 
 7   Division Name            23467 non-null  object
 8   Department Name          23467 non-null  object
 9   Class Name               23467 non-null  object
dtypes: int64(5), object(5)
memory usage: 2.0+ MB


In [14]:
pdf['Title'] = pdf['Title'].astype(str)
pdf['Review Text'] = pdf['Review Text'].astype(str)
pdf['Rating'] = pdf['Rating'].astype(int)
pdf['Division Name'] = pdf['Division Name'].astype(str)
pdf['Department Name'] = pdf['Department Name'].astype(str)
pdf['Class Name'] = pdf['Class Name'].astype(str)

In [15]:
data = spark.createDataFrame(pdf)

In [16]:
data.show(5)

+-----------+---+--------------------+--------------------+------+---------------+-----------------------+--------------+---------------+----------+
|Clothing ID|Age|               Title|         Review Text|Rating|Recommended IND|Positive Feedback Count| Division Name|Department Name|Class Name|
+-----------+---+--------------------+--------------------+------+---------------+-----------------------+--------------+---------------+----------+
|        767| 33|                 nan|Absolutely wonder...|     4|              1|                      0|     Initmates|       Intimate| Intimates|
|       1080| 34|                 nan|Love this dress! ...|     5|              1|                      4|       General|        Dresses|   Dresses|
|       1077| 60|Some major design...|I had such high h...|     3|              0|                      0|       General|        Dresses|   Dresses|
|       1049| 50|    My favorite buy!|I love, love, lov...|     5|              1|                      0|

In [17]:
data.printSchema()

root
 |-- Clothing ID: long (nullable = true)
 |-- Age: long (nullable = true)
 |-- Title: string (nullable = true)
 |-- Review Text: string (nullable = true)
 |-- Rating: long (nullable = true)
 |-- Recommended IND: long (nullable = true)
 |-- Positive Feedback Count: long (nullable = true)
 |-- Division Name: string (nullable = true)
 |-- Department Name: string (nullable = true)
 |-- Class Name: string (nullable = true)



In [18]:
data.columns

['Clothing ID',
 'Age',
 'Title',
 'Review Text',
 'Rating',
 'Recommended IND',
 'Positive Feedback Count',
 'Division Name',
 'Department Name',
 'Class Name']

In [19]:
data.count()

23481

# Preprocessing

In [20]:
data.filter(data['Title'].isNull()).count()

0

In [21]:
data.filter(data['Review Text'].isNull()).count()

0

In [22]:
data.filter(data['Recommended IND'].isNull()).count()

0

In [23]:
data.filter(data['Positive Feedback Count'].isNull()).count()

0

In [24]:
data.filter(data['Rating'].isNull()).count()

0

In [25]:
data.dropna(how= 'any', subset=['Title', 'Review Text', 'Recommended IND', 'Positive Feedback Count', 'Rating']).count()

23481

In [26]:
data = data.drop_duplicates()

# Process

In [27]:
#data_processed = data.select(['Review Text', 'Rating'])
data_processed = data.select(['Title', 'Review Text', 'Recommended IND', 'Positive Feedback Count', 'Rating'])

In [28]:
data_processed.show()

+--------------------+--------------------+---------------+-----------------------+------+
|               Title|         Review Text|Recommended IND|Positive Feedback Count|Rating|
+--------------------+--------------------+---------------+-----------------------+------+
|                 nan|                 nan|              1|                      0|     5|
|No slip and total...|I just received t...|              0|                      0|     2|
|           A winner!|What a well-desig...|              1|                      6|     5|
|                 nan|This top is full ...|              1|                      1|     5|
|Darling top- runs...|I am a solid size...|              1|                      0|     4|
| Perfect work pants!|I love these pant...|              1|                      0|     5|
|Looks cute - but ...|This jacket looke...|              0|                      4|     2|
|    Great fall dress|I tried this dres...|              1|                      0|     5|

In [29]:
# Pretty Clear Diference
data.groupby('Rating').mean().show()

+------+-----------------+-----------------+-----------+--------------------+----------------------------+
|Rating| avg(Clothing ID)|         avg(Age)|avg(Rating)|avg(Recommended IND)|avg(Positive Feedback Count)|
+------+-----------------+-----------------+-----------+--------------------+----------------------------+
|     5|913.9147978642258|43.58657513348589|        5.0|   0.998093058733791|           2.303432494279176|
|     1|916.0202140309156|43.70868014268728|        1.0|0.019024970273483946|          3.4625445897740783|
|     3|928.0585365853658|42.15470383275262|        3.0|  0.4142857142857143|          3.1414634146341465|
|     2|925.0095907928388|42.62723785166241|        2.0| 0.06010230179028133|          3.3222506393861893|
|     4|921.4336945812807|42.88453201970443|        4.0|  0.9668965517241379|          2.4061083743842366|
+------+-----------------+-----------------+-----------+--------------------+----------------------------+



In [30]:
data_processed.groupby('Rating').count().show()

+------+-----+
|Rating|count|
+------+-----+
|     5|13110|
|     1|  841|
|     3| 2870|
|     2| 1564|
|     4| 5075|
+------+-----+



In [31]:
data_processed = data_processed.withColumn('label', col('Rating')-lit(1))

In [32]:
data_processed.show()

+--------------------+--------------------+---------------+-----------------------+------+-----+
|               Title|         Review Text|Recommended IND|Positive Feedback Count|Rating|label|
+--------------------+--------------------+---------------+-----------------------+------+-----+
|                 nan|                 nan|              1|                      0|     5|    4|
|No slip and total...|I just received t...|              0|                      0|     2|    1|
|           A winner!|What a well-desig...|              1|                      6|     5|    4|
|                 nan|This top is full ...|              1|                      1|     5|    4|
|Darling top- runs...|I am a solid size...|              1|                      0|     4|    3|
| Perfect work pants!|I love these pant...|              1|                      0|     5|    4|
|Looks cute - but ...|This jacket looke...|              0|                      4|     2|    1|
|    Great fall dress|I tried 

# Feature Transformations

In [33]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from pyspark.ml.feature import CountVectorizer, IDF, StringIndexer

#rating_to_num = StringIndexer(inputCol = 'Rating', outputCol='label')

tokenizer_Title = Tokenizer(inputCol='Title', outputCol='token_Title')
stopremove_Title = StopWordsRemover(inputCol='token_Title', outputCol='stop_tokens_Title')
count_vec_Title = CountVectorizer(inputCol='stop_tokens_Title', outputCol='c_vec_Title')
idf_Title = IDF(inputCol='c_vec_Title', outputCol='tf_idf_Title')

tokenizer_review = Tokenizer(inputCol='Review Text', outputCol='token_text')
stopremove_review = StopWordsRemover(inputCol='token_text', outputCol='stop_tokens_text')
count_vec_review = CountVectorizer(inputCol='stop_tokens_text', outputCol='c_vec_text')
idf_review = IDF(inputCol='c_vec_text', outputCol='tf_idf_text')

In [34]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

In [35]:
clean_up = VectorAssembler(inputCols=['tf_idf_Title', 'tf_idf_text', 'Recommended IND', 'Positive Feedback Count'],
                           outputCol='features')

# Model

## Naive Bayes

In [36]:
from pyspark.ml.classification import NaiveBayes

In [37]:
nb = NaiveBayes()

## Logistic Regression

In [38]:
from pyspark.ml.classification import LogisticRegression

In [39]:
lg = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)

## Random Forest

In [40]:
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(labelCol="label",
                         featuresCol="features",
                         numTrees = 500,
                         maxDepth = 5,
                         maxBins = 64)

## Decision Tree

In [41]:
# Import the Decision Tree Classifier class
from pyspark.ml.classification import DecisionTreeClassifier
# Create a classifier object and fit to the training data
tree = DecisionTreeClassifier(featuresCol='features',
                             labelCol='label',
                              predictionCol='prediction'
                             )

## Pipeline

In [42]:
from pyspark.ml import Pipeline

In [43]:
data_prep_pipe = Pipeline(stages = [
                                    tokenizer_Title,
                                    tokenizer_review,
                                    stopremove_Title,
                                    stopremove_review,
                                    count_vec_Title,
                                    count_vec_review,
                                    idf_Title,
                                    idf_review,
                                    clean_up])

In [44]:
cleaner = data_prep_pipe.fit(data_processed)

In [45]:
clean_data = cleaner.transform(data_processed)

## Training and Evaluation!

In [46]:
clean_data = clean_data.select(['label', 'features'])

In [47]:
clean_data.show(10)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|    4|(43462,[0,6043,43...|
|    1|(43462,[4,150,349...|
|    4|(43462,[745,5923,...|
|    4|(43462,[0,5918,59...|
|    3|(43462,[10,33,215...|
|    4|(43462,[7,53,138,...|
|    1|(43462,[3,45,86,1...|
|    4|(43462,[1,4,35,59...|
|    0|(43462,[4,121,591...|
|    0|(43462,[1587,2138...|
+-----+--------------------+
only showing top 10 rows



In [48]:
clean_data.show(10)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|    4|(43462,[0,6043,43...|
|    1|(43462,[4,150,349...|
|    4|(43462,[745,5923,...|
|    4|(43462,[0,5918,59...|
|    3|(43462,[10,33,215...|
|    4|(43462,[7,53,138,...|
|    1|(43462,[3,45,86,1...|
|    4|(43462,[1,4,35,59...|
|    0|(43462,[4,121,591...|
|    0|(43462,[1587,2138...|
+-----+--------------------+
only showing top 10 rows



In [49]:
(training, testing) = clean_data.randomSplit([0.7, 0.3])

## resample data

In [50]:
training.groupby('label').count().show()

+-----+-----+
|label|count|
+-----+-----+
|    0|  585|
|    1| 1080|
|    3| 3547|
|    2| 2006|
|    4| 9091|
+-----+-----+



In [51]:
training.printSchema()

root
 |-- label: long (nullable = true)
 |-- features: vector (nullable = true)



=> Need to resample data

In [52]:
df_0 = training.filter(col("label") == 0)
df_1 = training.filter(col("label") == 1)
df_2 = training.filter(col("label") == 2)
df_3 = training.filter(col("label") == 3)
df_4 = training.filter(col("label") == 4)
ratio_0 = int(df_4.count() / df_0.count())
ratio_1 = int(df_4.count() / df_1.count())
ratio_2 = int(df_4.count() / df_2.count())
ratio_3 = int(df_4.count() / df_3.count())

print("ratio df_4/df_0: {}" .format(ratio_0))
print("ratio df_4/df_1: {}" .format(ratio_1))
print("ratio df_4/df_2: {}" .format(ratio_2))
print("ratio df_4/df_3: {}" .format(ratio_3))

ratio df_4/df_0: 15
ratio df_4/df_1: 8
ratio df_4/df_2: 4
ratio df_4/df_3: 2


In [53]:
# resample df_0
a0 = range(ratio_0)
# duplicate the minority rows
oversampled_df_0 = df_0.withColumn( "dummy", explode(array([lit(x) for x in a0]))).drop( 'dummy' )
# combine both oversampled minority rows and previous majority rows
combined_df = df_4.unionAll(oversampled_df_0)
combined_df.show(10)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|    4|(43462,[0,5911,59...|
|    4|(43462,[0,5918,59...|
|    4|(43462,[0,6043,43...|
|    4|(43462,[1,4,35,59...|
|    4|(43462,[1,9,5912,...|
|    4|(43462,[1,12,104,...|
|    4|(43462,[1,17,88,6...|
|    4|(43462,[1,126,656...|
|    4|(43462,[1,175,591...|
|    4|(43462,[1,196,591...|
+-----+--------------------+
only showing top 10 rows



In [54]:
combined_df.groupBy("label").count().show()

+-----+-----+
|label|count|
+-----+-----+
|    0| 8775|
|    4| 9091|
+-----+-----+



In [55]:
# resample df_1
a1 = range(ratio_1)
# duplicate the minority rows
oversampled_df_1 = df_1.withColumn( "dummy", explode(array([lit(x) for x in a1]))).drop( 'dummy' )
# combine both oversampled minority rows and previous majority rows
combined_df = combined_df.unionAll(oversampled_df_1)
combined_df.show(10)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|    4|(43462,[0,5911,59...|
|    4|(43462,[0,5918,59...|
|    4|(43462,[0,6043,43...|
|    4|(43462,[1,4,35,59...|
|    4|(43462,[1,9,5912,...|
|    4|(43462,[1,12,104,...|
|    4|(43462,[1,17,88,6...|
|    4|(43462,[1,126,656...|
|    4|(43462,[1,175,591...|
|    4|(43462,[1,196,591...|
+-----+--------------------+
only showing top 10 rows



In [56]:
combined_df.groupBy("label").count().show()

+-----+-----+
|label|count|
+-----+-----+
|    0| 8775|
|    1| 8640|
|    4| 9091|
+-----+-----+



In [57]:
# resample df_2
a2 = range(ratio_2)
# duplicate the minority rows
oversampled_df_2 = df_2.withColumn( "dummy", explode(array([lit(x) for x in a2]))).drop( 'dummy' )
# combine both oversampled minority rows and previous majority rows
combined_df = combined_df.unionAll(oversampled_df_2)
combined_df.show(10)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|    4|(43462,[0,5911,59...|
|    4|(43462,[0,5918,59...|
|    4|(43462,[0,6043,43...|
|    4|(43462,[1,4,35,59...|
|    4|(43462,[1,9,5912,...|
|    4|(43462,[1,12,104,...|
|    4|(43462,[1,17,88,6...|
|    4|(43462,[1,126,656...|
|    4|(43462,[1,175,591...|
|    4|(43462,[1,196,591...|
+-----+--------------------+
only showing top 10 rows



In [58]:
combined_df.groupBy("label").count().show()

+-----+-----+
|label|count|
+-----+-----+
|    0| 8775|
|    1| 8640|
|    2| 8024|
|    4| 9091|
+-----+-----+



In [59]:
# resample df_3
a3 = range(ratio_3)
# duplicate the minority rows
oversampled_df_3 = df_3.withColumn( "dummy", explode(array([lit(x) for x in a3]))).drop( 'dummy' )
# combine both oversampled minority rows and previous majority rows
combined_df = combined_df.unionAll(oversampled_df_3)
combined_df.show(10)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|    4|(43462,[0,5911,59...|
|    4|(43462,[0,5918,59...|
|    4|(43462,[0,6043,43...|
|    4|(43462,[1,4,35,59...|
|    4|(43462,[1,9,5912,...|
|    4|(43462,[1,12,104,...|
|    4|(43462,[1,17,88,6...|
|    4|(43462,[1,126,656...|
|    4|(43462,[1,175,591...|
|    4|(43462,[1,196,591...|
+-----+--------------------+
only showing top 10 rows



In [60]:
combined_df.groupBy("label").count().show()

+-----+-----+
|label|count|
+-----+-----+
|    0| 8775|
|    1| 8640|
|    3| 7094|
|    2| 8024|
|    4| 9091|
+-----+-----+



## Naive Bayes

In [64]:
#rating_predictor = nb.fit(training)
rating_predictor = nb.fit(combined_df)

In [65]:
test_results = rating_predictor.transform(testing)

In [66]:
test_results.show(10)

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|    0|(43462,[4,121,591...|[-2174.7452628747...|[1.0,9.6225287301...|       0.0|
|    0|(43462,[150,5763,...|[-1522.3944271800...|[4.27483438019073...|       2.0|
|    1|(43462,[2,55,75,2...|[-1226.7503483654...|[9.38330595708897...|       2.0|
|    1|(43462,[41,84,591...|[-824.66035147305...|[5.39869532272193...|       2.0|
|    1|(43462,[41,517,59...|[-2365.9288810936...|[4.80153887396154...|       1.0|
|    2|(43462,[0,5917,59...|[-1258.4198614939...|[0.99344659802346...|       0.0|
|    2|(43462,[3,107,242...|[-1948.0193540750...|[3.07388872938400...|       2.0|
|    2|(43462,[3,5911,59...|[-775.47364937069...|[3.30677413379999...|       4.0|
|    2|(43462,[8,99,5912...|[-631.35714778913...|[5.66126545005195...|       3.0|
|    2|(43462,[1

In [67]:
test_results.groupBy('label', 'prediction').count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    3|       0.0|   15|
|    4|       0.0|    5|
|    1|       3.0|  101|
|    3|       2.0|  220|
|    1|       0.0|   33|
|    2|       4.0|  116|
|    0|       1.0|   78|
|    3|       1.0|   42|
|    4|       4.0| 3187|
|    0|       4.0|   27|
|    3|       4.0|  642|
|    4|       2.0|  133|
|    1|       2.0|  201|
|    4|       3.0|  671|
|    2|       0.0|   34|
|    2|       2.0|  305|
|    4|       1.0|   23|
|    3|       3.0|  609|
|    0|       0.0|   44|
|    1|       4.0|   42|
+-----+----------+-----+
only showing top 20 rows



## Logistic Regression

In [69]:
#rating_predictor2 = lg.fit(training)
rating_predictor2 = lg.fit(combined_df)

In [70]:
test_results2 = rating_predictor2.transform(testing)

In [71]:
test_results2.groupBy('label', 'prediction').count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    3|       0.0|   12|
|    1|       3.0|   28|
|    4|       0.0|    2|
|    3|       2.0|  213|
|    1|       0.0|   78|
|    2|       4.0|  143|
|    0|       1.0|   83|
|    3|       1.0|   49|
|    4|       4.0| 3514|
|    0|       4.0|   29|
|    3|       4.0|  809|
|    4|       2.0|   89|
|    1|       2.0|  160|
|    4|       3.0|  399|
|    2|       0.0|   74|
|    2|       2.0|  310|
|    4|       1.0|   15|
|    3|       3.0|  445|
|    0|       0.0|   85|
|    1|       4.0|   51|
+-----+----------+-----+
only showing top 20 rows



## Random Forest

In [ ]:
#rating_predictor3 = rf.fit(training)
rating_predictor3 = rf.fit(combined_df)

In [ ]:
test_results3 = rating_predictor3.transform(testing)

In [ ]:
test_results3.groupBy('label', 'prediction').count().show()

## Decision Tree

In [61]:
#rating_predictor4 = tree.fit(training)
rating_predictor4 = tree.fit(combined_df)

In [62]:
test_results4 = rating_predictor4.transform(testing)

In [63]:
test_results4.groupBy('label', 'prediction').count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    3|       0.0|   43|
|    4|       0.0|    5|
|    1|       3.0|    2|
|    3|       2.0|   84|
|    1|       0.0|  362|
|    2|       4.0|  313|
|    0|       1.0|   23|
|    3|       1.0|   10|
|    4|       4.0| 3916|
|    0|       4.0|    7|
|    3|       4.0| 1333|
|    4|       2.0|   67|
|    1|       2.0|    7|
|    4|       3.0|   29|
|    2|       0.0|  389|
|    2|       2.0|   39|
|    4|       1.0|    2|
|    3|       3.0|   58|
|    0|       0.0|  223|
|    1|       4.0|   23|
+-----+----------+-----+
only showing top 20 rows



## Evaluation

In [68]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [72]:
acc_val = MulticlassClassificationEvaluator()
acc = acc_val.evaluate(test_results)
print('Accuracy of model at rating predicting was: {}'.format(acc))

Accuracy of model at rating predicting was: 0.5926347632930749


In [73]:
acc_val2 = MulticlassClassificationEvaluator()
acc2 = acc_val2.evaluate(test_results2)
print('Accuracy of model at rating predicting was: {}'.format(acc2))

Accuracy of model at rating predicting was: 0.6147768547398929


In [ ]:
acc_val3 = MulticlassClassificationEvaluator()
acc3 = acc_val3.evaluate(test_results3)
print('Accuracy of model at rating predicting was: {}'.format(acc3))

In [74]:
acc_val4 = MulticlassClassificationEvaluator()
acc4 = acc_val4.evaluate(test_results4)
print('Accuracy of model at rating predicting was: {}'.format(acc4))

Accuracy of model at rating predicting was: 0.5115773520775087


# Prediction

In [75]:
pdf_new = pd.read_excel('womens-ecommerce-clothing-reviews/Womens_Clothing_E_Commerce_Reviews.xlsx', 
                               sheet_name='new_reviews', index_col=0)

In [76]:
pdf_new.head()

,Clothing ID,Age,Title,Review Text,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,1077,53,Dress looks like it's made of cheap material,Dress runs small esp where the zipper area run...,0,14,General,Dresses,Dresses
1,862,66,Cute top,Nice top. armholes are a bit oversized but as ...,1,2,General,Tops,Knits
2,1080,31,Underwhelmed,Was really excited for this dress but should h...,0,1,General,Dresses,Dresses
3,936,35,Absolutely perfect,If you are going for a ridiculously high price...,0,9,General,Tops,Sweaters
4,872,35,Cute comfy casual,I saw this online and immediately purchased th...,1,0,General,Tops,Knits


In [77]:
pdf_new['Title'] = pdf['Title'].astype(str)
pdf_new['Review Text'] = pdf['Review Text'].astype(str)
pdf_new['Rating'] = pdf['Rating'].astype(int)
pdf_new['Division Name'] = pdf['Division Name'].astype(str)
pdf_new['Department Name'] = pdf['Department Name'].astype(str)
pdf_new['Class Name'] = pdf['Class Name'].astype(str)

In [78]:
data_new = spark.createDataFrame(pdf_new)

In [79]:
data_processed_new = data_new.select(['Title', 'Review Text', 'Recommended IND', 'Positive Feedback Count', 'Rating'])

In [80]:
data_processed_new = data_processed_new.withColumn('label', col('Rating')-lit(1))

In [81]:
clean_data_new = cleaner.transform(data_processed_new)

### Use logistic Regression to predict new rating

In [82]:
new_results = rating_predictor2.transform(clean_data_new)

In [83]:
new_results = new_results.withColumn('prediction_rating', col('prediction')+lit(1))

In [84]:
new_results.select('prediction_rating').show()

+-----------------+
|prediction_rating|
+-----------------+
|              5.0|
|              5.0|
|              3.0|
|              5.0|
|              5.0|
+-----------------+

